In [113]:
import pyspark
from pyspark import SparkContext
sc = SparkContext('local[4]')

# Load data

In [114]:
url = "pokemon.csv"
from pyspark import SparkFiles
from pyspark.sql import Row, SQLContext
from pyspark.sql.functions import concat, col, lit
sc.addFile(url)
sqlContext = SQLContext(sc)

In [115]:
df = sqlContext.read.csv(SparkFiles.get("pokemon.csv"), header=True, inferSchema= True)

# Data info (2)

In [116]:
df.printSchema()

root
 |-- abilities: string (nullable = true)
 |-- against_bug: double (nullable = true)
 |-- against_dark: double (nullable = true)
 |-- against_dragon: double (nullable = true)
 |-- against_electric: double (nullable = true)
 |-- against_fairy: double (nullable = true)
 |-- against_fight: double (nullable = true)
 |-- against_fire: double (nullable = true)
 |-- against_flying: double (nullable = true)
 |-- against_ghost: double (nullable = true)
 |-- against_grass: double (nullable = true)
 |-- against_ground: double (nullable = true)
 |-- against_ice: double (nullable = true)
 |-- against_normal: double (nullable = true)
 |-- against_poison: double (nullable = true)
 |-- against_psychic: double (nullable = true)
 |-- against_rock: double (nullable = true)
 |-- against_steel: double (nullable = true)
 |-- against_water: double (nullable = true)
 |-- attack: integer (nullable = true)
 |-- base_egg_steps: integer (nullable = true)
 |-- base_happiness: integer (nullable = true)
 |-- bas

In [117]:
df.show(5)

+--------------------+-----------+------------+--------------+----------------+-------------+-------------+------------+--------------+-------------+-------------+--------------+-----------+--------------+--------------+---------------+------------+-------------+-------------+------+--------------+--------------+----------+------------+--------------+-------+-----------------+--------+---+---------------------+----------+---------------+--------------+---------+----------+-----+-----+------+---------+----------+------------+
|           abilities|against_bug|against_dark|against_dragon|against_electric|against_fairy|against_fight|against_fire|against_flying|against_ghost|against_grass|against_ground|against_ice|against_normal|against_poison|against_psychic|against_rock|against_steel|against_water|attack|base_egg_steps|base_happiness|base_total|capture_rate| classfication|defense|experience_growth|height_m| hp|        japanese_name|      name|percentage_male|pokedex_number|sp_attack|sp_

In [118]:
df.count()

801

In [119]:
df.select('attack', 'defense').show(5)

+------+-------+
|attack|defense|
+------+-------+
|    49|     49|
|    62|     63|
|   100|    123|
|    52|     43|
|    64|     58|
+------+-------+
only showing top 5 rows



In [120]:
df.describe('attack').show()

+-------+-----------------+
|summary|           attack|
+-------+-----------------+
|  count|              801|
|   mean|77.85767790262172|
| stddev|32.15882023801748|
|    min|                5|
|    max|              185|
+-------+-----------------+



In [121]:
# How many are there of each type?
df.groupBy("type1").count().sort("count", ascending=False).show()

+--------+-----+
|   type1|count|
+--------+-----+
|   water|  114|
|  normal|  105|
|   grass|   78|
|     bug|   72|
| psychic|   53|
|    fire|   52|
|    rock|   45|
|electric|   39|
|  poison|   32|
|  ground|   32|
|    dark|   29|
|fighting|   28|
|  dragon|   27|
|   ghost|   27|
|   steel|   24|
|     ice|   23|
|   fairy|   18|
|  flying|    3|
+--------+-----+



# Combine 2 columns (3)

In [122]:
# Combine the columns and see how many BothTypes there are
df = df.withColumn("BothTypes",concat(col("type1"), lit('-'), col("type2")))
df.groupBy("BothTypes").count().sort("count", ascending=False).show()

+--------------+-----+
|     BothTypes|count|
+--------------+-----+
|          null|  384|
| normal-flying|   26|
|  grass-poison|   14|
|    bug-flying|   13|
|    bug-poison|   11|
|  water-ground|    9|
|  water-flying|    7|
|    rock-water|    6|
|psychic-flying|    6|
| steel-psychic|    6|
| fire-fighting|    6|
|  grass-flying|    6|
|   fire-flying|    6|
|   rock-ground|    6|
|     bug-grass|    6|
| psychic-fairy|    6|
|   grass-fairy|    5|
| water-psychic|    5|
|   dark-flying|    5|
|   ghost-grass|    5|
+--------------+-----+
only showing top 20 rows



In [123]:
# Then there come a lot of missing values

In [124]:
# Are there more or less types of each each generation
df.crosstab('type1', 'generation').sort('type1_generation').show()

+----------------+---+---+---+---+---+---+---+
|type1_generation|  1|  2|  3|  4|  5|  6|  7|
+----------------+---+---+---+---+---+---+---+
|             bug| 12| 10| 12|  8| 18|  3|  9|
|            dark|  0|  5|  4|  3| 13|  3|  1|
|          dragon|  3|  0|  7|  3|  7|  4|  3|
|        electric|  9|  6|  4|  7|  7|  3|  3|
|           fairy|  2|  5|  0|  1|  0|  9|  1|
|        fighting|  7|  2|  4|  2|  7|  3|  3|
|            fire| 12|  8|  6|  5|  8|  8|  5|
|          flying|  0|  0|  0|  0|  1|  2|  0|
|           ghost|  3|  1|  4|  6|  5|  4|  4|
|           grass| 12|  9| 12| 13| 15|  5| 12|
|          ground|  8|  3|  6|  4|  9|  0|  2|
|             ice|  2|  4|  6|  3|  6|  2|  0|
|          normal| 22| 15| 18| 17| 17|  4| 12|
|          poison| 14|  1|  3|  6|  2|  2|  4|
|         psychic|  8|  7|  8|  7| 14|  3|  6|
|            rock|  9|  4|  8|  6|  6|  8|  4|
|           steel|  0|  2|  9|  3|  4|  4|  2|
|           water| 28| 18| 24| 13| 17|  5|  9|
+------------

In [125]:
# Does the avg attack depend on the type of Pokemon
df.groupby('type1').agg({'attack': 'mean'}).sort('avg(attack)', ascending=False).show()

+--------+------------------+
|   type1|       avg(attack)|
+--------+------------------+
|  dragon| 106.4074074074074|
|fighting| 99.17857142857143|
|  ground|           94.8125|
|   steel| 93.08333333333333|
|    rock| 90.66666666666667|
|    dark| 87.79310344827586|
|    fire|              81.5|
|  normal| 75.16190476190476|
|   grass| 73.76923076923077|
|   water| 73.30701754385964|
|     ice| 73.30434782608695|
|   ghost| 72.74074074074075|
|  poison|          72.65625|
|electric| 70.82051282051282|
|     bug|            70.125|
|  flying| 66.66666666666667|
| psychic| 65.56603773584905|
|   fairy|62.111111111111114|
+--------+------------------+



# Add a column based on another column(4)

In [126]:
# Add attack and special attack
df = df.withColumn("full_attack", col("attack") + col('sp_attack'))

In [127]:
df.select('attack', 'sp_attack', 'full_attack').show(5)

+------+---------+-----------+
|attack|sp_attack|full_attack|
+------+---------+-----------+
|    49|       65|        114|
|    62|       80|        142|
|   100|      122|        222|
|    52|       60|        112|
|    64|       80|        144|
+------+---------+-----------+
only showing top 5 rows



In [129]:
# Look if the results are still the same does the type depend on the full attack
df.groupby('type1').agg({'full_attack': 'mean'}).sort('avg(full_attack)', ascending=False).show()

+--------+------------------+
|   type1|  avg(full_attack)|
+--------+------------------+
|  dragon|             196.0|
|    fire|169.23076923076923|
|   steel|165.79166666666666|
|    dark|162.31034482758622|
|electric|158.35897435897436|
| psychic|158.16981132075472|
|   ghost| 155.1851851851852|
|    rock|153.86666666666667|
|     ice| 150.7391304347826|
|  flying|150.66666666666666|
|fighting|149.28571428571428|
|   grass| 148.0897435897436|
|   water| 147.3684210526316|
|  ground|            146.75|
|   fairy|143.61111111111111|
|  poison|         134.21875|
|  normal|132.14285714285714|
|     bug|126.77777777777777|
+--------+------------------+



# Remove full attack <= 100 and missing values (5)

In [130]:
new_df = df.filter(df.full_attack > 100).na.drop()

In [131]:
new_df.select('full_attack','attack','sp_attack').describe().show()

+-------+------------------+------------------+-----------------+
|summary|       full_attack|            attack|        sp_attack|
+-------+------------------+------------------+-----------------+
|  count|               274|               274|              274|
|   mean|164.45985401459853| 86.44525547445255|78.01459854014598|
| stddev| 41.70373699201894|29.023905697702805|28.94987088394722|
|    min|               102|                30|               15|
|    max|               300|               185|              170|
+-------+------------------+------------------+-----------------+



In [132]:
new_df.select('full_attack').show(10)

+-----------+
|full_attack|
+-----------+
|        114|
|        142|
|        222|
|        263|
|        135|
|        165|
|        110|
|        215|
|        151|
|        167|
+-----------+
only showing top 10 rows



# Question (6): Based on some statistics (attack defence, sp attack , sp defence , height, hp, speed, weight) can you predict type?

In [133]:
sc.stop()